<a href="https://colab.research.google.com/github/kokchun/Machine-learning-AI22/blob/main/Lecture_code/L4-Regularization.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; for interacting with the code


---
# Regularized linear models

- Ridge regression
- LASSO
- ElasticNet
---

This is the lecture note for **regularized linear models**

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives a brief introduction to regularized linear models. I encourage you to read further about regularized linear models. </p>

Read more:

- [Regularized linear models medium](https://medium.com/analytics-vidhya/regularized-linear-models-in-machine-learning-d2a01a26a46)
- [Ridge regression wikipedia](https://en.wikipedia.org/wiki/Ridge_regression)
- [Tikhonov regularization wikipedia](https://en.wikipedia.org/wiki/Tikhonov_regularization)
- [Lasso regression wikipedia](https://en.wikipedia.org/wiki/Lasso_(statistics))
- [Korsvalidering](https://sv.wikipedia.org/wiki/Korsvalidering)
- [Cross validation](https://machinelearningmastery.com/k-fold-cross-validation/)
- [Scoring parameter sklearn](https://scikit-learn.org/stable/modules/model_evaluation.html)
- [ISLRv2 pp 198-205](https://www.statlearning.com/)
---


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

#plt.style.use("seaborn-white")

---
## Data preparation

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

df = pd.read_csv("../data/Advertising.csv", index_col=0)
X, y = df.drop("Sales", axis = 1), df["Sales"]

# in exercise 2 Polynomial regression you've found the elbow in degree 4, as the error increases after that
# however to be safe and we assume that the model shouldn't have too many interactions between different features, I will choose 3
# please try with 4 and see how your evaluation score differs

# transform features
model_polynomial = PolynomialFeatures(3, include_bias=False)
polynomial_features = model_polynomial.fit_transform(X)

# important to not forget 
X_train, X_test, y_train, y_test = train_test_split(polynomial_features, y, test_size=0.33, random_state=42)

# from 3 features we've featured engineered 19 with degree 3 polynom (x1, x1^2, x1^3, x1x2x3, ....)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((134, 19), (66, 19), (134,), (66,))

---
## Feature standardization
Remove sample mean and divide by sample standard deviation 

$X' = \frac{X-\mu}{\sigma}$

LASSO, Ridge and Elasticnet regression that we'll use later require that the data is scaled.


In [4]:
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train) # calculates parametes sigma and mu based on X_train and transforms X_transform
scaled_X_test = scaler.transform(X_test) # uses calculated mu and sigma to tranform X_test

print(f"Scaled X_train mean {scaled_X_train.mean():.2f}, std {scaled_X_train.std():.2f}")
print(f"Scaled X_test mean {scaled_X_test.mean():.2f}, std {scaled_X_test.std():.2f}")


Scaled X_train mean -0.00, std 1.00
Scaled X_test mean -0.12, std 1.12


---
## Regularization techniques - thikonov regularization - L2-regularization)

Problem with overfitting was discussed in previous lecture. When model is too complex, data noisy and dataset is too small the model picks up patterns in the noise. The output of a linear regression is the weighted sum: 
$y = \theta_0 + \theta_1x_1 + \theta_2x_2 + \ldots + \theta_nx_n$ , where the weights $\theta_i$ represents the importance of the $ith$ feature. Want to constrain the weights associated with noise, through regularization. We do this by adding a regularization term to the cost function used in training the model. Note that the cost function for evaluation now will differ from the training.

<p class = "alert alert-info" role="alert"><b>Note</b> most regularization model requires scaling of data </p>

---
### Ridge regression 
Also called Tikhonov regularization or $\ell_2$ regularization.

$C(\vec{\theta}) = MSE(\vec{\theta}) + \lambda \frac{1}{2}\sum_{i=1}^n \theta_i^2$

where $\lambda \ge 0$ is the ridge parameter or the penalty term, which reduces variance by increasing bias. Observe that the sum starts from 1, so the bias term $\theta_0$ is not affected by $\lambda$. Therefore by the larger the $\lambda$ the more $\theta_i, i = {1,2,\ldots}$ causes higher error. As variance is decreasing and bias increasing, the model fits worse to the training datas noise and generalizes better.

From the closed form OLS solution to ridge regression, we see that $\lambda = 0$ gives us the normal equation for linear regression: 

$\hat{\vec{\theta}} = (X^TX + \lambda I)^{-1}X^T\vec{y}$



In [5]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error


def ridge_regression(X_test, penalty=0): #X_test dvs det vi vill predicta på, när funktioien används så är det med scaled_X_test
    # alpha = 0 should give linear regression
    # alhpa is Ridge same as lambda in theory, i.e. penalty term. sklearn has chosen alpha to generalize their API
    model_ridge = Ridge(alpha=penalty) 
    model_ridge.fit(scaled_X_train, y_train)
    y_pred = model_ridge.predict(X_test)
    return y_pred


y_pred = ridge_regression(scaled_X_test, penalty = 0) # since panalty = 0 this is polynomial regression


MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE


(0.3748516441217865, 0.2650465950553601, 0.5148267621786576)

In [6]:
# check with linear regression -> RMSE very similar!

from sklearn.linear_model import LinearRegression

# polynomial regression
model_linear = LinearRegression()
model_linear.fit(scaled_X_train, y_train)
y_pred = model_linear.predict(scaled_X_test)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE

(0.3748516441217832, 0.26504659505536404, 0.5148267621786614)

In [7]:
# testing with penalty (own guess of 0.01)
# to hyperparameter tune need validation data

y_pred = ridge_regression(scaled_X_test, penalty = 0.01)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE

(0.3716840615001231, 0.26202411194681025, 0.5118829084339603)

---
## Lasso regression - L1
Lasso - Least Absolute Shrinkage and Selection Operator or $\ell_1$ regularization. Cost function for Lasso is: 

$C(\vec{\theta}) = MSE(\vec{\theta}) + \lambda\sum_{i=1}^n |\theta_i|$

It sets least important features to zero, when $\lambda$ sufficiently large. This is practically feature selection. 

Note that in Lasso regression it is important to have scaled your data

In [8]:
from sklearn.linear_model import Lasso
model_lasso = Lasso(alpha = .1)
model_lasso.fit(scaled_X_train, y_train)
y_pred = model_lasso.predict(scaled_X_test)

MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE

(0.5735346450114956, 0.6168472080645071, 0.7853962108799017)

### k-fold cross-validation

One strategy to choose the best hyperparameter alpha is to take the training part of the data and 
1. shuffle dataset randomly
2. split into k groups
3. for each group -> take one test, the rest training -> fit the model -> predict on test -> get evaluation metric
4. take the mean of the evaluation metrics
5. choose the parameters and train on the entire training dataset

Repeat this process for each alpha, to see which yielded lowest RMSE. k-fold cross-validation: 
- good for smaller datasets
- fair evaluation, as a mean of the evaluation metric for all k groups is calculated
- expensive to compute as it requires k+1 times of training

---
### Ridge regression

In [17]:
from sklearn.linear_model import RidgeCV # ridge regression with cross-validation
from sklearn.metrics import SCORERS

#print(SCORERS.keys())
# negative because sklearn uses convention of higher return values are better
# cv is k, k-fold
model_ridgeCV = RidgeCV(alphas = [.0001, .001, .01, .1, .5, 1, 5, 10], scoring = "neg_mean_squared_error") # passing in a list of alphas
model_ridgeCV.fit(scaled_X_train, y_train)
print(model_ridgeCV.alpha_)


0.1


In [18]:
# best alpha is 0.1
# it seems that linear regression outperformed ridge regression in this case
# however it could depend on the distribution of the train|test data, so using alpha = 0.1 is more robust here
y_pred = model_ridgeCV.predict(scaled_X_test)

MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE

(0.4343075766531668, 0.31763359449891565, 0.5635899169599432)

In [11]:
model_ridgeCV.coef_

array([ 5.84681185,  0.52142086,  0.71689997, -6.17948738,  3.75034058,
       -1.36283352, -0.08571128,  0.08322815, -0.34893776,  2.16952446,
       -0.47840838,  0.68527348,  0.63080799, -0.5950065 ,  0.61661989,
       -0.31335495,  0.36499629,  0.03328145, -0.13652471])

### Lasso regression - L1

In [12]:
from sklearn.linear_model import LassoCV

# it is trying 100 different alphas along regularization path epsilon
model_lassoCV = LassoCV(n_alphas = 200, cv=5) # cv is k in k-fold, model chooses 200 alphas, could also pass in a list of alphas to be tested
model_lassoCV.fit(scaled_X_train, y_train)
print(f"alpha = {model_lassoCV.alpha_}") # penelty (landa in theory) found through 5-fold cross validation model


alpha = 0.004968802520343366


c:\Users\DorotaBjöörn-AI22GBG\.virtualenvs\Machine-Learning-Dorota-Bjoorn-ugpTJiOr\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.548e-01, tolerance: 3.571e-01
  model = cd_fast.enet_coordinate_descent(


In [13]:
# we notice that many coefficients have been set to 0 using Lasso
# it has selected some features for us 
# 0. means that those features have been removed
#5.196 is beta_0
# 19th values since had 19 features after feature engineering
model_lassoCV.coef_

array([ 5.11468536,  0.42127203,  0.28896055, -4.63391705,  3.48972093,
       -0.390611  ,  0.        ,  0.        ,  0.        ,  1.24969939,
       -0.        ,  0.        ,  0.13795383, -0.01666923,  0.        ,
        0.        ,  0.10974819,  0.        ,  0.0458376 ])

In [14]:
# predicing with lassoCV with calculated alpha
y_pred = model_lassoCV.predict(scaled_X_test)

MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE

(0.46802072322691207, 0.3410150044071009, 0.5839648999786724)

### Elastic net

Elastic net is a combination of both Ridge l2-regularization and Lasso l1-regularization. The cost function to be minimized for elastic net is: 

$$C(\vec{\theta}) = MSE(\vec{\theta}) + \lambda\left(\alpha\sum_{i=1}^n |\theta_i| + \frac{1-\alpha}{2}\sum_{i=1}^n \theta_i^2\right)$$

, where $\alpha$ here determines the ratio for $\ell_1$ or $\ell_2$ regularization.

In [19]:
from sklearn.linear_model import ElasticNetCV

# note that alpha here is lambda in the theory
# l1_ratio is alpha in the theory
model_elastic = ElasticNetCV(l1_ratio=[.1, .5, .7, .8, .9, .95, .99, 1], eps = 0.001, n_alphas = 100, max_iter=int(1e4))
model_elastic.fit(scaled_X_train, y_train)
print(f"L1 ratio: {model_elastic.l1_ratio_}") # this would remove ridge and pick Lasso regression entirely
print(f"alpha {model_elastic.alpha_}") # same alphs as when we used ridge

# L1-ratio  = 1 so LASSO regression

L1 ratio: 1.0
alpha 0.004968802520343366


In [20]:
y_pred = model_elastic.predict(scaled_X_test)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE
# note that the result is same for Lasso regression which is expected

(0.46291883026932984, 0.33467924600222104, 0.5785146895301977)

---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---
